In [81]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import BertTokenizer
import csv
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel
import time
import datetime
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup
import sys
import argparse
import os
from collections import defaultdict
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from typing import List, Dict
from torch.utils.data import Dataset
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from typing import List
import tqdm
import csv
import json
import faiss  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv("results.covid_dataset.all.tsv", sep = "\t")

In [3]:
ids, sents = df["sentence_id"].tolist(), df["sentence_text"].tolist()

In [4]:
len(sents)

2318939

In [5]:
sents[0]

'Sequences in bold indicate codons requiring multiple nucleotide substitutions .'

In [6]:
sents[5]

'These analytic weights were then multiplied by the appropriate adjustment factors as described above .'

In [11]:
class BertEncoder(object):
    
    def __init__(self, device = 'cpu'):
        
        self.tokenizer = BertTokenizer.from_pretrained('/media/shauli/Elements/current_projects/AI2/Clustering/scibert_scivocab_uncased/vocab.txt')
        self.model = BertModel.from_pretrained('/media/shauli/Elements/current_projects/AI2/Clustering/scibert_scivocab_uncased/')
            
        self.model.eval()
        self.model.to(device)
        self.device = device
        self.pad_token = self.tokenizer.convert_tokens_to_ids([self.tokenizer.pad_token])[0]
        
    def tokenize_and_pad(self, texts: List[str]):
        
        indexed_texts = [self.tokenizer.encode(text, add_special_tokens=True, max_length = 512) for text in texts] #
        max_len = max(len(text) for text in indexed_texts)
        indexed_texts = [text + [self.pad_token] * (max_len - len(text)) for text in indexed_texts]
        idx_tensor = torch.LongTensor(indexed_texts).to(self.device)
        att_tensor = idx_tensor != self.pad_token
        
        return idx_tensor, att_tensor
    
    def encode(self, sentences: List[str], sentence_ids: List[str], batch_size: int, strategy: str = "cls"):
        assert len(sentences) == len(sentence_ids)
        
        with open("output.jsonl", "w", encoding = "utf-8") as f:
            
            for batch_idx in tqdm.tqdm_notebook(range(0, len(sentences), batch_size), total = len(sentences)//batch_size):
            
                batch_sents = sentences[batch_idx: batch_idx + batch_size]
                batch_ids = sentence_ids[batch_idx: batch_idx + batch_size]
                assert len(batch_sents) == len(batch_ids)
                
                idx, att = self.tokenize_and_pad(batch_sents)
            
                with torch.no_grad():
                    outputs = self.model(idx)
                    last_hidden = outputs[0]
                
                    if strategy == "cls":
                        h = last_hidden[:, 0, ...]
                    elif strategy == "mean":
                        h = torch.mean(last_hidden, axis = 1)
                    elif strategy == "median":
                        h = torch.median(last_hidden, axis = 1).values
                    elif strategy == "max":
                        h = torch.max(last_hidden, axis = 1).values
                    elif strategy == "min":
                        h = torch.min(last_hidden, axis = 1).values
            
                batch_np = h.detach().cpu().numpy()
                assert len(batch_np) == len(batch_sents)
                
                sents_states_ids = zip(batch_sents, batch_np, batch_ids)
                for sent, vec, sent_id in sents_states_ids:
                    
                    vec_str = " ".join(["%.4f" % x for x in vec])
                    sent_dict = {"text": sent, "vec": vec_str, "id": sent_id}
                    f.write(json.dumps(sent_dict) + "\n")

In [12]:
encoder = BertEncoder("cuda")

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [14]:
h = encoder.encode(sents, ids, batch_size = 32, strategy = "cls")

In [170]:
n = 50000
data = []

with open("output.jsonl", "r", encoding = "utf-8") as f:
    i = 0
    for line in f:
            data_dict = eval(line)
            data_dict["vec"] = np.array([float(x) for x in data_dict["vec"].split(" ")]).astype("float32")
            data.append(data_dict)
            i += 1
            if i > n: break

In [171]:
vecs_sample = np.array([d["vec"] for d in data])
sents_sample =[d["text"] for d in data]
pca = PCA(n_components = 0.98)
vecs_sample_pca = pca.fit_transform(vecs_sample).astype("float32")

index = faiss.IndexFlatIP(vecs_sample_pca.shape[1])
index.add(np.ascontiguousarray(vecs_sample_pca))

In [199]:
k = 15
spread_idx = np.array([i for i,sent in enumerate(sents_sample) if "spread" in sent])
spread_mean = np.median(vecs_sample[spread_idx], axis = 0)
spread_mean_pca = pca.transform([spread_mean])
D, I = index.search(np.ascontiguousarray(spread_mean_pca), 10)

In [200]:
spread_idx[:10]

array([ 27,  29,  37,  38,  87,  91,  94, 146, 185, 230])

In [201]:
sents[37]

'These findings highlight a new explanatory pattern for the risk of HPAI and indicate that , in addition to agro-environmental factors , anthropogenic factors play an important role in the spread of H5N1 .'

In [202]:
for i in I.squeeze()[1:]:
    print(sents_sample[i])
    print("===========================================================")

The parasitism of vegetal organisms also had a great impact in the history of the humanity .
None of the infected animals became ill , despite the reported virulence of the challenge viruses .
The olfactory pathway is amenable to surgical intervention at the level of the nasal epithelium , the olfactory nerve , or the olfactory bulb .
New targets for novel anti-infective drugs need to be identified , particularly in the light of the emergence of drug resistant strains .
Indeed , the CHWs stressed the importance of the various incentives in carrying out their duties .
HPNV-specific transcripts were detected in the cells in the intestine , indicating that this virus replicates in the intestinal tract .
Interestingly , the procedure involved the delivery of the biological preparation through intranasal inoculation , a common route of modern mucosal vaccination .
this study evaluated the role of pmn during a non-fatal pulmonary coronavirus infection in the natural host .
This would allow t

In [168]:
sents[1]

'Poisson regression with analytic weights , offsets , and robust variance estimation was used to implement the above extrapolation and standardization procedures for estimating seasonal incidence and 95 % confidence intervals ( CIs ) [ 17 ] [ 18 ] [ 19 ] .'